In [1]:
import box_count_method as bcm
import tkinter as tk
from tkinter import filedialog

import pandas as pd
import pandas_bokeh
from bokeh.io import output_file, output_notebook
from bokeh.models import DatetimeTickFormatter, ranges
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.palettes import linear_palette, Turbo256
pd.set_option('plotting.backend', 'pandas_bokeh')
output_notebook(hide_banner=True)

root = tk.Tk()
root.withdraw()

folder = filedialog.askdirectory(title='Please select the image / working directory')

phaseContent = bcm.phaseContent(folder)

Checking files in the directory
processing file #10
processing file #20
processing file #30
processing file #40
processing file #50
processing file #60
processing file #70
processing file #80
processing file #90
Found 100 images
mean std deviation of phase_00_percent
mean std deviation of phase_01_percent
mean std deviation of phase_02_percent


In [33]:
def getLineDiagram(title, plot_width=1200, legend_location='top_center'):
    p = figure(title=title, plot_width=plot_width, plot_height=600)
    p.left[0].formatter.use_scientific = False
    p.xaxis.axis_label = 'number#'
    p.yaxis.axis_label = 'sigma'

    i = 0
    colors= linear_palette(Turbo256,len(phaseContent.phase_list))
    for phase in phaseContent.phase_list:
        key = 'phase_{:02d}_percent'.format(i)
        stdev = phaseContent.stDev_stDev[key]
        mean = phaseContent.stDev_mean[key]
        positive = list(map(float.__add__, mean, stdev))
        negative = list(map(float.__sub__, mean, stdev))
        p.line(list(range(len(stdev))), positive, line_color=colors[i],line_dash='dashed', line_width=2, legend_label=key)
        p.line(list(range(len(stdev))), negative, line_color=colors[i],line_dash='dashed', line_width=2, legend_label=key)
        p.line(list(range(len(stdev))), mean, line_color=colors[i], line_width=2, legend_label=key)
        i += 1

    p.legend.location = legend_location
    show(p)

In [34]:
getLineDiagram('Test')